## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать параметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pydrive.drive import GoogleDrive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':"1j-DhO_XD5EqzOqVSR4Wz1kNzhLmpeTkZ"})   # замените id на id своего файла
downloaded.GetContentFile('women-clothing-accessories.csv')
sms_spam = pd.read_csv('women-clothing-accessories.csv', delimiter='\t', usecols=[0, 1])
sms_spam = sms_spam.sample(10000)

In [ ]:
!pip install pymorphy2

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from copy import copy 
from pymorphy2 import MorphAnalyzer
from sklearn.metrics import * 
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import string

pymorphy2_analyzer = MorphAnalyzer()
nltk.download('stopwords')
nltk.download('punkt')

def fit_transform(vectorizer):
  return vectorizer, vectorizer.fit_transform(my_x_train)

def bayes(vectorizer, vectorizer_x_train):
  my_clf = MultinomialNB()
  my_clf.fit(vectorizer_x_train, my_y_train)  
  vectorizer_x_test = vectorizer.transform(my_x_test)
  my_pred = my_clf.predict(vectorizer_x_testв_pred, output_dict=True), vectorizer, my_pred

def processing(data):
  sentences = []
  for sentence in data:
    sentence = sentence.lower()
    for ch in '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\t\n':
      sentence = sentence.replace(ch,"")
    sentence = re.sub(r'\d', '', sentence)
    words = word_tokenize(sentence)
    for i in range(len(words)):
      words[i] = pymorphy2_analyzer.parse(words[i])[0].normal_form
    sentences.append(" ".join(words))
  return sentences

sms_spam.review = processing(sms_spam.review)
my_x_train, my_x_test, my_y_train, my_y_test = train_test_split(sms_spam.review, sms_spam.sentiment, train_size = 0.7)

print('word')
maximum = (0,0)
for i in range(1, 6):
  for j in range(i + 1, 6):
    vectorizer = CountVectorizer(ngram_range=(i,j), analyzer="word")
    count_vectorizer, count_vectorizer_x_train = fit_transform(vectorizer)
    a = bayes(count_vectorizer, count_vectorizer_x_train)
    if a[0]['accuracy'] > maximum[0]:
      maximum = (a[0]['accuracy'], a)
print("ngram_range:", str(maximum[1][1]))
print(classification_report(my_y_test, maximum[1][2], output_dict=False))

print('char')
maximum = (0,0)
for i in range(1, 6):
  for j in range(i + 1, 6):
    vectorizer = CountVectorizer(ngram_range=(i,j), analyzer="char")
    count_vectorizer, count_vectorizer_x_train = fit_transform(vectorizer)
    a = bayes(count_vectorizer, count_vectorizer_x_train)
    if a[0]['accuracy'] > maximum[0]:
      maximum = (a[0]['accuracy'], a)
print("ngram_range:", str(maximum[1][1]))
print(classification_report(my_y_test, maximum[1][2], output_dict=False))

print('tfid')
maximum = (0,0)
for i in range(1, 6):
  for j in range(i + 1, 6):
    vectorizer = TfidfVectorizer(ngram_range=(i,j))
    count_vectorizer, count_vectorizer_x_train = fit_transform(vectorizer)
    a = bayes(count_vectorizer, count_vectorizer_x_train)
    if a[0]['accuracy'] > maximum[0]:
      maximum = (a[0]['accuracy'], a)
print("ngram_range:", str(maximum[1][1]))
print(classification_report(my_y_test, maximum[1][2], output_dict=False))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
word


NameError: ignored

## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
sentence = 'mom washed the frame'
def first_two_letters(x):
  return re.findall(r'\b\w{2}', x)

first_two_letters(sentence)


['mo', 'wa', 'th', 'fr']

### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
result = re.split('\.', 'Mom washed the frame. Mom washed the frame. Mom washed the frame. Mom washed the frame. \
 Mom washed the frame. Mom washed the frame. Mom washed the frame. Mom washed the frame.', maxsplit=3)
print(result)

['Mom washed the frame', ' Mom washed the frame', ' Mom washed the frame', ' Mom washed the frame.  Mom washed the frame. Mom washed the frame. Mom washed the frame. Mom washed the frame.']


### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
result = re.sub(r'\d', 'DIG', 'I\'ve bought 3 apples and 75 bananas')
print(result)

I've bought DIG apples and DIGDIG bananas


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
result = re.sub(r'http[s]?://([^ ])*', '', 'The best site is https://piazza.com/class/kt725hi4odv52p?cid=50 and i like it')
print(result)

The best site is  and i like it


### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
prog = re.compile('[А-Яа-яё\-]{4,}')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'больше', 'больше', 'слов', 'Что-то']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
domens = re.compile('@[A-Za-z0-9]*.[A-Za-z]{2,4}')
domens.findall("abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz")

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']